In [2]:
import tensorflow as tf
from tf_agents.environments import suite_gym
from tf_agents.environments.wrappers import TimeLimit
from tf_agents.environments.tf_py_environment import TFPyEnvironment
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.utils import common
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.environments import tf_py_environment
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.trajectories import trajectory  # Add this line

C:\Users\kaan-\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Define the environment
env_name = 'CartPole-v1'
environment = suite_gym.load(env_name)
environment = TFPyEnvironment(TimeLimit(environment, duration=1000))

# Define the Q-network
fc_layer_params = (100,)

q_net = q_network.QNetwork(
    environment.observation_spec(),
    environment.action_spec(),
    fc_layer_params=fc_layer_params)

# Define the DQN agent
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    time_step_spec=environment.time_step_spec(),
    action_spec=environment.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

# Define the replay buffer
replay_buffer_capacity = 10000
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=environment.batch_size,
    max_length=replay_buffer_capacity)

# Define the data collection
collect_steps_per_iteration = 1
collect_op = dynamic_step_driver.DynamicStepDriver(
    environment,
    agent.collect_policy,
    observers=[replay_buffer.add_batch],
    num_steps=collect_steps_per_iteration).run()

# Define the dataset
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=64,
    num_steps=2).prefetch(3)

# Define the training
num_iterations = 1000
batch_size = 32

train_op = common.function(agent.train)
agent.train_step_counter.assign(0)


Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


<tf.Variable 'UnreadVariable' shape=() dtype=int32, numpy=0>

In [4]:
# Train the agent
for _ in range(num_iterations):
    # Collect a few steps using collect_policy and save to the replay buffer.
    for _ in range(collect_steps_per_iteration):
        # Collect data into the replay buffer
        time_step = environment.current_time_step()
        action_step = agent.collect_policy.action(time_step)
        next_time_step = environment.step(action_step.action)
        traj = trajectory.from_transition(time_step, action_step, next_time_step)

        # Add trajectory to replay buffer
        replay_buffer.add_batch(traj)

    # Check if the replay buffer has enough data
    if replay_buffer.num_frames().numpy() >= batch_size:
        # Sample a batch of data from the replay buffer
        experience, unused_info = next(iter(dataset))

        # Train the agent
        train_info = agent.train(experience)

        # Print training status
        step = agent.train_step_counter.numpy()
        if step % 1000 == 0:
            print('Step: {}, Loss: {}'.format(step, train_info.loss))
    else:
        print("Replay buffer does not have enough data for training.")

Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have enough data for training.
Replay buffer does not have 

In [5]:
# Test the agent
num_episodes = 10  # You can adjust the number of episodes for testing

for episode in range(num_episodes):
    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
        action_step = agent.policy.action(time_step)
        time_step = environment.step(action_step.action)
        episode_return += time_step.reward.numpy()

    print('Episode {}: Total Return: {}'.format(episode + 1, episode_return))

Episode 1: Total Return: [76.]
Episode 2: Total Return: [88.]
Episode 3: Total Return: [79.]
Episode 4: Total Return: [57.]
Episode 5: Total Return: [63.]
Episode 6: Total Return: [87.]
Episode 7: Total Return: [83.]
Episode 8: Total Return: [84.]
Episode 9: Total Return: [75.]
Episode 10: Total Return: [78.]


In [14]:
# Test the agent with rendering
num_episodes = 3

for episode in range(num_episodes):
    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
        action_step = agent.policy.action(time_step)
        time_step = environment.step(action_step.action)
        episode_return += time_step.reward.numpy()

        # Render the environment
        environment.render(mode='human')
    print('Episode {}: Total Return: {}'.format(episode + 1, episode_return))

print('closing')

# Close the environment after testing
# environment.close()

Episode 1: Total Return: [64.]
Episode 2: Total Return: [63.]
Episode 3: Total Return: [78.]
closing


: 